In [3]:
import pandas as pd

In [4]:
df = pd.read_parquet('/workspaces/mlops_track/yellow_tripdata_2023-01.parquet', engine='pyarrow')

In [5]:
print(df.columns)
print(df.shape)

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')
(3066766, 19)


There are 19 columns in the dataframe

In [6]:
df.head

<bound method NDFrame.head of          VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0               2  2023-01-01 00:32:10   2023-01-01 00:40:36              1.0   
1               2  2023-01-01 00:55:08   2023-01-01 01:01:27              1.0   
2               2  2023-01-01 00:25:04   2023-01-01 00:37:49              1.0   
3               1  2023-01-01 00:03:48   2023-01-01 00:13:25              0.0   
4               2  2023-01-01 00:10:29   2023-01-01 00:21:19              1.0   
...           ...                  ...                   ...              ...   
3066761         2  2023-01-31 23:58:34   2023-02-01 00:12:33              NaN   
3066762         2  2023-01-31 23:31:09   2023-01-31 23:50:36              NaN   
3066763         2  2023-01-31 23:01:05   2023-01-31 23:25:36              NaN   
3066764         2  2023-01-31 23:40:00   2023-01-31 23:53:00              NaN   
3066765         2  2023-01-31 23:07:32   2023-01-31 23:21:56              NaN  

In [7]:
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)



In [8]:

df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
td = df.duration.iloc[0]


df['duration'] = df['duration'].apply(lambda td:td.total_seconds() / 60 )

In [9]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'duration'],
      dtype='object')

In [10]:
duration_std = df.duration.std()
print(duration_std)

42.59435124195458


In [11]:
print(df.duration.min())
print(df.duration.max())
print(df.duration.mean())

-29.2
10029.183333333332
15.668995167330452


In [12]:
record_count = len(df)
print(record_count)

3066766


In [13]:
df = df[(df['duration'] > 1) & (df['duration'] < 60)]

filtered_record = len(df)
outlier_percentage = ((record_count - filtered_record) /record_count) * 100
print(outlier_percentage)

1.8885366539214274


98% of the records was left after removing the ouliers

In [14]:
df = df.dropna()

In [15]:
from sklearn.feature_extraction import DictVectorizer

In [16]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df.loc[:, categorical] = df[categorical].astype(str)

/tmp/ipykernel_51272/4215808272.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['161' '43' '48' ... '79' '68' '236']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, categorical] = df[categorical].astype(str)
/tmp/ipykernel_51272/4215808272.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['141' '237' '238' ... '246' '238' '262']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, categorical] = df[categorical].astype(str)


In [17]:
train_dicts = df[categorical + numerical].to_dict(orient = 'records')

In [18]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [19]:
print(X_train.shape[1])

515


In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [21]:
y_train = df['duration']

In [22]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [23]:
y_pred_train = model.predict(X_train)
mse_train = mean_squared_error(y_train, y_pred_train)
print(mse_train)

rmse_train = np.sqrt(mse_train)
print(rmse_train)

58.11954283438762
7.623617437567787


Applying model to the validation dataset

In [24]:
df_val = pd.read_parquet('/workspaces/mlops_track/yellow_tripdata_2023-02.parquet', engine='pyarrow')

In [25]:
df_val.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [26]:
df_val.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')

In [27]:
df_val.tpep_dropoff_datetime = pd.to_datetime(df_val.tpep_dropoff_datetime)
df_val.tpep_pickup_datetime = pd.to_datetime(df_val.tpep_pickup_datetime)


In [28]:

df_val['duration'] = df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']
td = df_val.duration.iloc[0]


df_val['duration'] = df_val['duration'].apply(lambda td:td.total_seconds() / 60 )

In [29]:
df_val =df_val.dropna()

In [30]:
categorical_val = ['PULocationID', 'DOLocationID']
numerical_val = ['trip_distance']

df_val.loc[:, categorical_val] = df[categorical_val].astype(str)


/tmp/ipykernel_51272/179942500.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['161' '43' '48' ... '79' '231' '249']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_val.loc[:, categorical_val] = df[categorical_val].astype(str)
/tmp/ipykernel_51272/179942500.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['141' '237' '238' ... '231' '144' '141']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_val.loc[:, categorical_val] = df[categorical_val].astype(str)


In [31]:
val_dicts = df_val[categorical_val + numerical_val].to_dict(orient = 'records')

In [32]:
dv_val = DictVectorizer()
X_val = dv.fit_transform(val_dicts)

In [33]:
print(X_val.shape[1])

517


In [37]:
y_pred_val = model.predict(X_val)
mse_val = mean_squared_error(y_val, y_pred_val)
print(mse_val)

rmse_val = np.sqrt(mse_val)
print(rmse_val)

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values